In [ ]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time
import matplotlib.pyplot as plt
import psutil
import gc
# Definir o timezone GMT+3
GMT_PLUS_3 = pytz.timezone('Etc/GMT-3')

# Definir parâmetros
symbol = "XAUUSD"
lot_size = 0.01  # Tamanho do lote ajustado para um valor comum
timeframe = mt5.TIMEFRAME_M15  # Timeframe de 15 minutos
timeframe_get_candle = mt5.TIMEFRAME_M1
n_candles = 10  # Número de candles para identificar pontos altos e baixos
spread = 2  # Spread para calcular SL
spread_tp =1
end_of_day_time = "23:59"  # Hora de zerar as operações no fim do dia
start_of_day_time = "01:00"  # Hora de início das operações no início do dia
start_date = datetime(2025, 4, 21, 0, 0, tzinfo=GMT_PLUS_3)
end_date = start_date + timedelta(days=5)  # 5 dias úteis

# Função para inicializar e conectar ao MetaTrader 5
def initialize_mt5():
    """
    Initialize MT5 connection using environment variables.
    
    Required environment variables:
    - MT5_ACCOUNT: Your MetaTrader 5 account number
    - MT5_PASSWORD: Your MT5 account password
    - MT5_SERVER: Your broker's MT5 server name
    
    Create a .env file in the project root with these variables.
    """
    import os
    from dotenv import load_dotenv
    
    # Load environment variables from .env file
    load_dotenv()
    
    if not mt5.initialize():
        print("Falha na inicialização / Initialization failed")
        return False
    
    # Get credentials from environment variables
    account = os.getenv('MT5_ACCOUNT')
    password = os.getenv('MT5_PASSWORD')
    server = os.getenv('MT5_SERVER')
    
    # Validate credentials
    if not all([account, password, server]):
        print("ERROR: Missing credentials. Please set MT5_ACCOUNT, MT5_PASSWORD, and MT5_SERVER in your .env file")
        print("ERRO: Credenciais faltando. Configure MT5_ACCOUNT, MT5_PASSWORD e MT5_SERVER no arquivo .env")
        mt5.shutdown()
        return False
    
    # Convert account to integer
    try:
        account = int(account)
    except ValueError:
        print("ERROR: MT5_ACCOUNT must be a number")
        mt5.shutdown()
        return False
    
    if not mt5.login(account, password, server):
        print(f"Falha ao conectar à conta {account}, erro: {mt5.last_error()}")
        print(f"Failed to connect to account {account}, error: {mt5.last_error()}")
        mt5.shutdown()
        return False
    else:
        print(f"Conectado à conta {account} / Connected to account {account}")
        return True
    
def monitor_memory():
    process = psutil.Process()
    memory_info = process.memory_info()
    print(f"Uso de memória: {memory_info.rss / (1024 * 1024)} MB")    

# Função para garantir que o símbolo está disponível
def ensure_symbol(symbol):
    selected = mt5.symbol_select(symbol, True)
    if not selected:
        print(f"Falha ao selecionar símbolo {symbol}")
        return False

    info = mt5.symbol_info(symbol)
    if not info:
        print(f"Informações do símbolo {symbol} não disponíveis")
        return False

    if not info.visible:
        if not mt5.symbol_select(symbol, True):
            print(f"Falha ao tornar o símbolo {symbol} visível")
            return False

    return True

# Função para obter dados históricos usando copy_rates_range
def get_historical_data(symbol, timeframe, start_time, end_time):
    rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)
    if rates is None or len(rates) == 0:
        raise ValueError(f"Nenhum dado retornado para o símbolo {symbol} no intervalo de {start_time} a {end_time}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')  # Não converter para UTC
    df['time'] = df['time'].dt.tz_localize(GMT_PLUS_3)  # Localizar diretamente como GMT+3
    return df

# Função para identificar pontos altos e baixos
def identify_points(df):
    if df.empty or 'close' not in df.columns:
        raise ValueError("DataFrame vazio ou faltando coluna 'close'")
    
    subset_df = df.iloc[:n_candles]
    first_high = subset_df['close'].max()
    first_low = subset_df['close'].min()
    return first_high, first_low

# Função para enviar ordens de compra
def send_buy_order(symbol, lot_size, sl, tp):
    price = mt5.symbol_info_tick(symbol).ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot_size,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": sl,
        "tp": tp,
        "deviation": 10,
        "magic": 234000,
        "comment": "Compra automática",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    print(f"Ordem de compra enviada: {result}")
    return result

# Função para enviar ordens de venda
def send_sell_order(symbol, lot_size, sl, tp):
    price = mt5.symbol_info_tick(symbol).bid
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot_size,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
        "tp": tp,
        "deviation": 10,
        "magic": 234000,
        "comment": "Venda automática",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    print(f"Ordem de venda enviada: {result}")
    return result

# # Função para plotar candles
# def plot_candles(df, zn_bot, zn_upper, cr_bot, cr_upper, n1_100_bot, n1_100_upper, n2_100_upper, n2_100_bot, n3_100_upper, n3_100_bot, filename, buy_signals=[], sell_signals=[]):
#     if df.empty:
#         raise ValueError("DataFrame vazio fornecido para plotagem")

#     df['time'] = df['time'].dt.tz_convert(GMT_PLUS_3)
#     today = datetime.now(GMT_PLUS_3)
#     last_monday = today - timedelta(days=today.weekday())
#     last_monday = last_monday.replace(hour=0, minute=0, second=0, microsecond=0)
#     last_friday = last_monday + timedelta(days=4, hours=23, minutes=59, seconds=59)
#     df_week = df[(df['time'] >= last_monday) & (df['time'] <= last_friday)]

#     plt.figure(figsize=(10, 6))
#     plt.plot(df_week['time'], df_week['close'], 'bo-', label='Close Price')
#     plt.axhline(zn_bot, color='yellow', linestyle='--', label='ZN Bot')
#     plt.axhline(zn_upper, color='yellow', linestyle='--', label='ZN Upper')
#     plt.axhline(cr_bot, color='black', linestyle='--', label='CR Bot')
#     plt.axhline(cr_upper, color='black', linestyle='--', label='CR Upper')
#     plt.axhline(n1_100_bot, color='red', linestyle='--', label='N1 100 Bot')
#     plt.axhline(n1_100_upper, color='green', linestyle='--', label='N1 100 Upper')
#     plt.axhline(n2_100_bot, color='red', linestyle='--', label='N2 100 Bot')
#     plt.axhline(n2_100_upper, color='green', linestyle='--', label='N2 100 Upper')
#     plt.axhline(n3_100_bot, color='red', linestyle='--', label='N3 100 Bot')
#     plt.axhline(n3_100_upper, color='green', linestyle='--', label='N3 100 Upper')

#     for signal in buy_signals:
#         plt.plot(signal[0], signal[1], 'g^', markersize=10, label='Buy Signal')
#     for signal in sell_signals:
#         plt.plot(signal[0], signal[1], 'rv', markersize=10, label='Sell Signal')

#     plt.legend()
#     plt.xlabel('Date')
#     plt.ylabel('Price')
#     plt.title('Candlestick Chart with Support and Resistance Lines')
#     plt.xticks(rotation=45)
#     plt.grid()
#     plt.savefig(filename)
#     plt.close()  # Fechar a figura para liberar memória

# Função para obter o último candle de 15 minutos
def get_last_candle(symbol, timeframe):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1)
    if rates is None or len(rates) == 0:
        raise ValueError(f"Não foi possível obter o último candle para o símbolo {symbol}")
    df = pd.DataFrame(rates)
    if df.empty:
        raise ValueError(f"DataFrame vazio retornado para o símbolo {symbol}")
    df['time'] = pd.to_datetime(df['time'], unit='s').dt.tz_localize(GMT_PLUS_3)
    return df.iloc[-1]

# Função para verificar se há posições abertas
def has_open_positions(symbol):
    positions = mt5.positions_get(symbol=symbol)
    return len(positions) > 0

def get_week_candles(start_date):
    start_timestamp = int(start_date.timestamp())
    end_date = start_date + timedelta(days=5)
    total_minutes = (end_date - start_date).total_seconds() / 60
    num_candles = int(total_minutes / 15)
    candles = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_candles)

    if candles is None:
        print("Error: ", mt5.last_error())
    else:
        df = pd.DataFrame(candles)
        if df.empty or 'time' not in df.columns or 'open' not in df.columns or 'close' not in df.columns:
            raise ValueError("DataFrame retornado está vazio ou faltam colunas necessárias")
        df['time'] = pd.to_datetime(df['time'], unit='s').dt.tz_localize(GMT_PLUS_3)
        df_filtered = df[(df['time'] >= start_date) & (df['time'] < end_date)]
        df_filtered.to_csv(f'x_{symbol}.csv')
    return df_filtered

def monitor_gc():
    gc.set_debug(gc.DEBUG_LEAK)
    gc.collect()
    for obj in gc.garbage:
        print(f"Objeto não coletado: {obj}")

def monitor_market():
    df = None
    while df is None or len(df) < n_candles:
        try:
            df = get_week_candles(start_date)
        except ValueError as e:
            print(e)
            df = None

        if df is None or len(df) < n_candles:
            print(f"Aguardando os primeiros {n_candles} candles...")
            time.sleep(60)

    first_high, first_low = identify_points(df)
    cr_upper = first_high
    cr_bot = first_low
    zn_upper = cr_upper + (cr_upper - cr_bot)
    zn_bot = cr_bot - (cr_upper - cr_bot)
    n1_50_upper = zn_upper + (zn_upper - cr_bot) / 2
    n1_100_upper = zn_upper + (zn_upper - cr_bot)
    n2_50_upper = zn_upper + (zn_upper - cr_bot) * 1.5
    n2_100_upper = zn_upper + (zn_upper - cr_bot) * 2
    n3_50_upper = zn_upper + (zn_upper - cr_bot) * 2.5
    n3_100_upper = zn_upper + (zn_upper - cr_bot) * 3

    n1_50_bot = zn_bot - (cr_upper - zn_bot) / 2
    n1_100_bot = zn_bot - (cr_upper - zn_bot)
    n2_50_bot = zn_bot - (cr_upper - zn_bot) * 1.5
    n2_100_bot = zn_bot - (cr_upper - zn_bot) * 2
    n3_50_bot = zn_bot - (cr_upper - zn_bot) * 2.5
    n3_100_bot = zn_bot - (cr_upper - zn_bot) * 3

    print(f"Variáveis calculadas: cr_upper={cr_upper}, cr_bot={cr_bot}, zn_upper={zn_upper}, zn_bot={zn_bot}")

    past_operations = log_past_operations(df, zn_bot, zn_upper, cr_upper, cr_bot, n1_100_bot, n1_100_upper, n2_100_bot, n2_100_upper)
    for op in past_operations:
        print(op)

    buy_signals = []
    sell_signals = []

    while True:
        current_time = datetime.now(GMT_PLUS_3)
        print(f'horario servidor mt5: {current_time}.')

        try:
            last_candle = get_last_candle(symbol, timeframe_get_candle)
            open_price = last_candle['open']
            close_price = last_candle['close']
            check_df = pd.DataFrame([last_candle])
            
            print(f"Verificando candle de {last_candle['time']}")
            print(f"Open price: {open_price}, Close price: {close_price}")
            print(f"zn_bot: {zn_bot}, zn_upper: {zn_upper}")
            print(f"n1_upper: {n1_100_upper}, n1_bot: {n1_100_bot}")

            if not has_open_positions(symbol):
                if open_price > zn_bot and close_price < zn_bot:
                    sl = cr_upper + spread
                    tp = n1_100_bot+spread_tp
                    print(f"Condição de venda atendida: Open={open_price}, Close={close_price}, SL={sl}, TP={tp}")
                    result = send_sell_order(symbol, lot_size, sl, tp)
                    if result.retcode != mt5.TRADE_RETCODE_DONE:
                        print(f"Erro ao enviar ordem de venda: {result.comment}")
                    else:
                        print(f"Ordem de venda enviada: SL={sl}, TP={tp}")
                        sell_signals.append((last_candle['time'], close_price))

                if open_price < zn_upper and close_price > zn_upper:
                    sl = cr_bot - spread
                    tp = n1_100_upper-spread_tp
                    print(f"Condição de compra atendida: Open={open_price}, Close={close_price}, SL={sl}, TP={tp}")
                    result = send_buy_order(symbol, lot_size, sl, tp)
                    if result.retcode != mt5.TRADE_RETCODE_DONE:
                        print(f"Erro ao enviar ordem de compra: {result.comment}")
                    else:
                        print(f"Ordem de compra enviada: SL={sl}, TP={tp}")
                        buy_signals.append((last_candle['time'], close_price))
            else:
                print(f"Posições abertas encontradas. Aguardando fechamento das posições para novas ordens.")

            levels = {
                "n1_50_bot": n1_50_bot, "n1_100_bot": n1_100_bot, "n2_50_bot": n2_50_bot, "n2_100_bot": n2_100_bot,
                "n1_50_upper": n1_50_upper, "n1_100_upper": n1_100_upper, "n2_50_upper": n2_50_upper, "n2_100_upper": n2_100_upper
            }

            for level, value in levels.items():
                if "bot" in level and df['low'].min() <= value:
                    print(f"Nível {level} ({value}) foi alcançado pela baixa.")
                elif "upper" in level and df['high'].max() >= value:
                    print(f"Nível {level} ({value}) foi alcançado pela alta.")

            # plot_candles(df, zn_bot, zn_upper, cr_bot, cr_upper, n1_100_bot, n1_100_upper, n2_100_upper, n2_100_bot, n3_100_upper, n3_100_bot, f"candlestick_chart_{symbol}.png", buy_signals, sell_signals)

            # Limpar DataFrames e listas
            del check_df,last_candle
            gc.collect()

        except ValueError as e:
            print(e)
            time.sleep(60)
            continue

        end_of_day = GMT_PLUS_3.localize(datetime.combine(current_time.date(), datetime.strptime(end_of_day_time, '%H:%M').time()))
        if current_time >= end_of_day:
            positions = mt5.positions_get(symbol=symbol)
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_BUY if pos.type == mt5.ORDER_TYPE_SELL else mt5.ORDER_TYPE_SELL,
                    "position": pos.ticket,
                    "price": mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_SELL else mt5.symbol_info_tick(symbol).ask,
                    "deviation": 10,
                    "magic": 234000,
                    "comment": "Encerramento automático",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }
                result = mt5.order_send(close_request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"Erro ao encerrar posição: {result.comment}")
                else:
                    print(f"Posição encerrada: ticket={pos.ticket}")

        print("Aguardando condições de mercado...")
        time.sleep(10)
        monitor_memory()
        # monitor_gc()
# Função para registrar operações que deveriam ter sido realizadas
def log_past_operations(df, zn_bot, zn_upper, cr_upper, cr_bot, n1_100_bot, n1_100_upper, n2_100_bot, n2_100_upper):
    if df.empty:
        raise ValueError("DataFrame vazio fornecido para log de operações passadas")
    if 'open' not in df.columns or 'close' not in df.columns or 'time' not in df.columns:
        raise ValueError("DataFrame fornecido para log de operações passadas está faltando colunas necessárias")
    
    operations = []
    for i in range(len(df)):
        open_price = df['open'].iloc[i]
        close_price = df['close'].iloc[i]
        time = df['time'].iloc[i]

        if open_price > zn_bot and close_price < zn_bot:
            operations.append(f"Venda deveria ter sido realizada: Open={open_price}, Close={close_price}, SL={cr_upper + spread}, TP={n1_100_bot}, as:{time}")
        if open_price < zn_upper and close_price > zn_upper:
            operations.append(f"Compra deveria ter sido realizada: Open={open_price}, Close={close_price}, SL={cr_bot - spread}, TP={n1_100_upper}, as:{time}")

    return operations

# Inicializar e conectar ao MetaTrader 5
if initialize_mt5():
    if ensure_symbol(symbol):
        monitor_market()
        mt5.shutdown()
    else:
        print(f"Não foi possível selecionar o símbolo {symbol}")
else:
    print("Não foi possível inicializar e conectar ao MetaTrader 5")


Conectado à conta [REDACTED]
Variáveis calculadas: cr_upper=3355.58, cr_bot=3338.25, zn_upper=3372.91, zn_bot=3320.92
Compra deveria ter sido realizada: Open=3368.77, Close=3373.77, SL=3336.25, TP=3407.5699999999997, as:2025-04-21 04:00:00+03:00
Compra deveria ter sido realizada: Open=3371.38, Close=3376.38, SL=3336.25, TP=3407.5699999999997, as:2025-04-21 04:30:00+03:00
horario servidor mt5: 2025-04-22 20:11:33.160101+03:00.
Verificando candle de 2025-04-22 20:11:00+03:00
Open price: 3390.88, Close price: 3396.37
zn_bot: 3320.92, zn_upper: 3372.91
n1_upper: 3407.5699999999997, n1_bot: 3286.26
Nível n1_50_upper (3390.24) foi alcançado pela alta.
Nível n1_100_upper (3407.5699999999997) foi alcançado pela alta.
Nível n2_50_upper (3424.8999999999996) foi alcançado pela alta.
Nível n2_100_upper (3442.2299999999996) foi alcançado pela alta.
Aguardando condições de mercado...
Uso de memória: 154.16015625 MB
horario servidor mt5: 2025-04-22 20:11:43.252720+03:00.
Verificando candle de 2025-04

KeyboardInterrupt: 